## TensorBoard使用

**SummaryWriter**

- add_image()

    - 功能
    
        记录图像
    
    - tag
    
        图像标签名
        
    - img_tensor
    
        图像数据，注意尺度 tensor的值范围在0-1之间，会自动乘以255，否则按原值显示
    
    - global_step
    
        x轴 通过拖动x轴获取图像
        
    - dataformats
    
        数据形式：CHW/HWC/HW
        
    

In [4]:
import os
import torch
import time
import torchvision.models as models
import torchvision.transforms as transforms
import torchvision.utils as vutils
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.nn.functional as F

In [2]:
# ----------------------------------- 3 image -----------------------------------
writer = SummaryWriter(comment='test_your_comment',
                       filename_suffix="_test_your_filename_suffix")

# img 1     random
fake_img = torch.randn(3, 512, 512)  # 0-1之间，乘以255以后显示
writer.add_image("fake_img", fake_img, 1)
time.sleep(1)

# img 2     ones
fake_img = torch.ones(3, 512, 512)
time.sleep(1)
writer.add_image("fake_img", fake_img, 2)

# img 3     1.1
fake_img = torch.ones(3, 512, 512) * 1.1    # 大于1了 所以按原值显示
time.sleep(1)
writer.add_image("fake_img", fake_img, 3)

# img 4     HW
fake_img = torch.rand(512, 512)
writer.add_image("fake_img", fake_img, 4, dataformats="HW")

# img 5     HWC
fake_img = torch.rand(512, 512, 3)
writer.add_image("fake_img", fake_img, 5, dataformats="HWC")  ##注意格式

writer.close()

**torchvision.utils.makes_grid**

- 功能：

    制作网格图像，一张图像显示多张图像，而不需要step进行选择
    
- tensor

    图像数据，B\*C\*H*W
    
- nrow

    行数，一列有多少图像
    
- padding

    图像间距（像素单位） 分隔图像用
    
- normalize

    是否将像素值标准化，标准化到0-255区间
    
- range

    标准化范围  将像素值限制在固定区间内，然后进行归一化到0-255之间
    
- scale_each

    是否单张图维度标准化
    
- pad_value

    padding的像素值


In [2]:
# ----------------------------------- 使用 make_grid 显示数据集图像-----------------------------------

import torchvision

writer = SummaryWriter(comment='test_your_comment',
                       filename_suffix="_test_your_filename_suffix")

transform_compose = transforms.Compose(
    [transforms.Resize((32, 64)),
     transforms.ToTensor()])
trainset = torchvision.datasets.MNIST(root='./data/MNIST',
                                        train=True,
                                        download=True,
                                        transform=transform_compose)
train_loader = torch.utils.data.DataLoader(trainset,
                                          batch_size=16,
                                          shuffle=True)
# 获取一个batch的图片显示
data_batch, label_batch = next(iter(train_loader))

img_grid = vutils.make_grid(data_batch,
                            nrow=4,
                            normalize=True,
                            scale_each=True)
# img_grid = vutils.make_grid(data_batch, nrow=4, normalize=False, scale_each=False)
writer.add_image("input img", img_grid, 0)
writer.close()

## 卷积核的可视化

In [4]:
# ----------------------------------- kernel visualization -----------------------------------

writer = SummaryWriter(comment='test_your_comment',
                       filename_suffix="_test_your_filename_suffix")

alexnet = models.alexnet(pretrained=True)  ##载入预训练模型

kernel_num = -1
vis_max = 1

for sub_module in alexnet.modules(): # alexnet.modules() 返回所有子模块
    if isinstance(sub_module, nn.Conv2d):
        kernel_num += 1
        if kernel_num > vis_max:
            break
        kernels = sub_module.weight
        c_out, c_int, k_w, k_h = tuple(kernels.shape) # 转为元组 c_out表示卷积核个数  c_int表示卷积核通道数
        
        '''
        2种可视化方法
        '''
        
        
        # 单个卷积核的依次可视化   通道数设为1 所以为黑白图像
        for o_idx in range(c_out):
            kernel_idx = kernels[o_idx, :, :, :].unsqueeze(1)  # make_grid需要 BCHW，这里拓展C维度 将c_in看作卷积核个数
            kernel_grid = vutils.make_grid(kernel_idx,
                                           normalize=True,
                                           scale_each=True,
                                           nrow=c_int)
            writer.add_image(
                '{}_Convlayer_split_in_channel'.format(kernel_num),
                kernel_grid,
                global_step=o_idx)
        
        # 所有卷积核直接可视化  通道设为3 所以为彩色图像
        kernel_all = kernels.view(-1, 3, k_h, k_w)  # 3, h, w
        kernel_grid = vutils.make_grid(kernel_all,
                                       normalize=True,
                                       scale_each=True,
                                       nrow=8)  # c, h, w
        writer.add_image('{}_all'.format(kernel_num),
                         kernel_grid,
                         global_step=322)

        print("{}_convlayer shape:{}".format(kernel_num, tuple(kernels.shape)))

writer.close()

0_convlayer shape:(64, 3, 11, 11)
1_convlayer shape:(192, 64, 5, 5)


## 特征图可视化

In [7]:
# ----------------------------------- feature map visualization -----------------------------------
from PIL import Image

writer = SummaryWriter(comment='test_your_comment',
                       filename_suffix="_test_your_filename_suffix")

# 数据
path_img = "./lena.png"  # your path to image
normMean = [0.49139968, 0.48215827, 0.44653124]
normStd = [0.24703233, 0.24348505, 0.26158768]

norm_transform = transforms.Normalize(normMean, normStd)
img_transforms = transforms.Compose(
    [transforms.Resize((224, 224)),
     transforms.ToTensor(), norm_transform])

img_pil = Image.open(path_img).convert('RGB')
if img_transforms is not None:
    img_tensor = img_transforms(img_pil)  # 对img的图像处理
img_tensor.unsqueeze_(0)  # chw --> bchw

# 模型
alexnet = models.alexnet(pretrained=True)

# forward
convlayer1 = alexnet.features[0]
# 获取特征图
fmap_1 = convlayer1(img_tensor)  # 调用call函数进行卷积计算

# 预处理
fmap_1.transpose_(0, 1)  # bchw=(1, 64, 55, 55) --> (64, 1, 55, 55)
fmap_1_grid = vutils.make_grid(fmap_1, normalize=True, scale_each=True, nrow=8)

# 显示特征图
writer.add_image('feature map in conv1', fmap_1_grid, global_step=322)
writer.close()

## 计算图以及模型显示

    add_graph()

- 功能

    可视化模型计算图
    
- model

    模型，nn.model()
    
- input_to_model

    输出给模型的数据，给定的是 shape
    
- verbose

    是否打印计算图结构信息，注意需要1.3.0环境
    

In [5]:
# 模型
class LeNet(nn.Module):
    def __init__(self, classes):
        super(LeNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, classes)

    def forward(self, x):
        out = F.relu(self.conv1(x))
        out = F.max_pool2d(out, 2)
        out = F.relu(self.conv2(out))
        out = F.max_pool2d(out, 2)
        out = out.view(out.size(0), -1)
        out = F.relu(self.fc1(out))
        out = F.relu(self.fc2(out))
        out = self.fc3(out)
        return out

    def initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.xavier_normal_(m.weight.data)
                if m.bias is not None:
                    m.bias.data.zero_()
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight.data, 0, 0.1)
                m.bias.data.zero_()

In [6]:
# ----------------------------------- 5 add_graph -----------------------------------
writer = SummaryWriter(comment='test_your_comment',
                       filename_suffix="_test_your_filename_suffix")

# 输入数据，保持维度正确
fake_img = torch.randn(1, 3, 32, 32)

lenet = LeNet(classes=2)

# 加入模型
writer.add_graph(lenet, fake_img)

writer.close()

    torch.summary
    
- 功能
    
    查看模型信息便于调试

- model

    pytorch模型
    
- input_size
    
    模型输入size
    
- batch_size

    batch_size  默认为-1 不需要设置
    
- device

    cuda cpu

In [8]:
print(torch.__version__)

1.4.0


In [8]:
from torchsummary import summary
print(summary(lenet, (1, 32, 32), device="cpu"))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 6, 28, 28]             456
            Conv2d-2           [-1, 16, 10, 10]           2,416
            Linear-3                  [-1, 120]          48,120
            Linear-4                   [-1, 84]          10,164
            Linear-5                    [-1, 2]             170
Total params: 61,326
Trainable params: 61,326
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.01
Forward/backward pass size (MB): 0.05
Params size (MB): 0.23
Estimated Total Size (MB): 0.30
----------------------------------------------------------------
None
